In [ ]:
# Get packages

import os
import pandas as pd
from matplotlib import pyplot as plt
import datetime

In [ ]:
# Read in case study data
os.chdir("/Users/jh/Downloads")
tx_df = pd.read_excel("Retail Category Forecasting.xlsx", sheet_name="Data")

In [ ]:
# View data
print(tx_df.head())
print("")

# How big is the dataset
print(f"Dataset dimensions: {tx_df.shape}")
print("")

# Data types
print("Column data types:")
print(tx_df.dtypes)
print("")

# Unique taxonomy count
print(f"Unique taxonomy count: {len(tx_df['taxonomy'].unique())}")
print("")

# Statistics of page_views
print("Statistics of page_views")
print(tx_df['page_views'].describe())
print("")

# Statistics of week_start_date
print("Statistics of week_start_date")
print(tx_df['week_start_date'].describe(datetime_is_numeric=True))
print("")

In [ ]:
# View total page views by week_start_date
pv_x_wsd = tx_df.groupby('week_start_date')['page_views'].sum().reset_index().sort_values('week_start_date')

plt.figure(figsize=(10,4))
ax = plt.axes()
ax.set_facecolor("whitesmoke")
plt.title('Total Page Views by Date')
plt.plot(pv_x_wsd['week_start_date'], pv_x_wsd['page_views'], linewidth=0.5)
plt.xticks(rotation = 45, fontsize=8)
plt.grid(color="white")
plt.show()

In [ ]:
print("PLP vs. CLP record counts:")
print(tx_df['page_type'].value_counts())

In [ ]:
# View total page views by week_start_date and page_type
pv_x_wsd_pt = tx_df.groupby(['week_start_date', 'page_type'])['page_views'].sum().reset_index().sort_values(['page_type', 'week_start_date'])


plt.figure(figsize=(10,4))
ax = plt.axes()
ax.set_facecolor("whitesmoke")
plt.title('Page Views by Date and CLP/PLP')
plt.plot(pv_x_wsd_pt[pv_x_wsd_pt['page_type'] == 'PLP']['week_start_date'], pv_x_wsd_pt[pv_x_wsd_pt['page_type'] == 'PLP']['page_views'], linewidth=0.5, color="blue")
plt.plot(pv_x_wsd_pt[pv_x_wsd_pt['page_type'] == 'CLP']['week_start_date'], pv_x_wsd_pt[pv_x_wsd_pt['page_type'] == 'CLP']['page_views'], linewidth=0.5, color="red")
plt.xticks(rotation = 45, fontsize=8)
plt.grid(color="white")
plt.legend(['PLP', 'CLP'])
plt.show()

In [ ]:
# See if there are any missing dates
# Get DF with unique week_start_date and lag 1 of week_start_date
wsd = pd.concat([pd.DataFrame(tx_df['week_start_date'].unique(), columns=['week_start_date']),
                 pd.DataFrame(tx_df['week_start_date'].unique(), columns=['week_start_date_lag']).shift(periods=1)],
                axis=1
               )

# Get days difference between each pair of week_start_date and week_start_date_lag
wsd['days_diff'] = wsd['week_start_date'] - wsd['week_start_date_lag']

# Look at the distribution of 'days_diff'
# If all values are 1, then no missing days, if any values are greater than 1, then there are missing days
print(wsd['days_diff'].value_counts())
# Days diff are all 1 days, the data as a whole has no missing days

# The label week_start_date does not make sense with daily observations
# A better label could be 'date'

In [ ]:
tx_wsd_df = tx_df[['taxonomy', 'week_start_date']].drop_duplicates()['taxonomy'].value_counts().reset_index()
tx_wsd_df.columns = ["taxonomy", "cnt_of_days"]
print(tx_wsd_df.sort_values('cnt_of_days').iloc[:20, :])
print(tx_wsd_df["cnt_of_days"].value_counts().sort_values(ascending=False))

In [ ]:
# Explore CLP vs. PLP
# How many taxonomies have both CLP and PLP page types
print(tx_df[['taxonomy', 'page_type']].drop_duplicates().groupby('taxonomy').count().sort_values('page_type', ascending=False))
print("")
# 4 taxonomies have CLP and PLP page_types

# Isolate taxonomies with 2 page_types
multi_pt_tx = tx_df[['taxonomy', 'page_type']].drop_duplicates().groupby('taxonomy').count().reset_index()
print(multi_pt_tx[multi_pt_tx['page_type'] > 1]['taxonomy'].to_list())
print("")

# Check if any of those 4 taxonomies have different page_type views on the same day
tmp_df = tx_df[['taxonomy', 'page_type', 'week_start_date']].drop_duplicates().groupby(['taxonomy', 'week_start_date']).count().sort_values('page_type', ascending=False)
print(tmp_df['page_type'].value_counts())
print("")

print(tx_df[['taxonomy', 'page_type']].drop_duplicates())

In [ ]:
# Create DF without page_type
tx_df2 = tx_df.groupby(['taxonomy', 'week_start_date'])['page_views'].sum().reset_index().rename(columns={'week_start_date': 'traffic_date'})
tx_df2.columns
print(tx_df2)

In [ ]:
# Look at 'flooring>garage flooring' taxonomy and plot
# 'flooring>garage flooring' does not have page_views on every date
tmp_df = tx_df2[tx_df2['taxonomy'] == 'flooring>garage flooring'].sort_values('traffic_date').reset_index(drop=True)
print(tmp_df)

# Some days have page_views = 0
# This taxonomy has missing dates
# Assumption: missing dates have page_views = 0

# View total page views by traffic_date
plt.figure(figsize=(10,4))
ax = plt.axes()
ax.set_facecolor("whitesmoke")
plt.title('Garage Flooring Page Views by Date')
plt.plot(tmp_df['traffic_date'], tmp_df['page_views'], linewidth=0.5)
plt.xticks(rotation = 45, fontsize=8)
plt.grid(color="white")
plt.show()

In [ ]:
# How many page_views = 0 across all taxonomies
pv_zero_cnt = tx_df2[tx_df2['page_views'] == 0].count()[0]
pv_gt_zero_cnt = tx_df2[tx_df2['page_views'] > 0].count()[0]

print(f"Count of page_views = 0: {pv_zero_cnt}")
print(f"Count of page_views > 0: {pv_gt_zero_cnt}")
print(f"Percent of page_views = 0: {pv_zero_cnt/tx_df2.shape[0]}")

In [ ]:
# Find first traffic_date where page_views > 0 per taxonomy
tx_dt1 = tx_df2[tx_df2['page_views'] > 0].groupby('taxonomy')['traffic_date'].min().reset_index().rename(columns={"traffic_date": "first_traffic_date"})

# Get all unique taxonomies and all unique traffic_dates
tx = pd.DataFrame(tx_df2['taxonomy'].drop_duplicates().reset_index(drop=True))
dts = pd.DataFrame(tx_df2['traffic_date'].drop_duplicates().reset_index(drop=True))

# Create a key to help with the equivalent of a cross join
tx['key'] = 0
dts['key'] = 0

# Create a dataset where all taxonomies have all traffic_dates from the first date where page_views > 0
tx_dts = tx.merge(dts, on="key", how="outer").drop(columns='key')
tx_dts = tx_dts.merge(tx_dt1, on="taxonomy", how="inner")
tx_dts = tx_dts[tx_dts['traffic_date'] >= tx_dts['first_traffic_date']].drop(columns='first_traffic_date')

# Merge tx_dts to tx_df2
tx_df3 = tx_dts.merge(tx_df2, on=["taxonomy", "traffic_date"], how="left")
print(tx_df3["page_views"].isna().value_counts())
print(tx_df3.shape[0])

# Many "missing" observations: 21K out of 138K

In [ ]:
# Add features to tx_df3 to facilitate computation of filter rules
missing_values_pct_threshold = 0.3
max_observations_threshold = 5

tx_df3['missing_values'] = 0
tx_df3.loc[tx_df3['page_views'].isna() == True, 'missing_values'] = 1

tx_df3['zero_values'] = 0
tx_df3.loc[tx_df3['page_views'] == 0, 'zero_values'] = 1

# Create DF to capture filter statistics at the taxonomy level
tx_filter_summary = tx_df3['taxonomy'].value_counts().reset_index().rename(columns={"index": "taxonomy", "taxonomy": "dt_cnt"})

# Add statistics to tx_filter_summary
tx_filter_summary = tx_filter_summary.merge(tx_df3.groupby('taxonomy')['page_views'].max().reset_index().rename(columns={"page_views": "max_pv"}), on="taxonomy", how="left")
tx_filter_summary = tx_filter_summary.merge(tx_df3.groupby('taxonomy')['missing_values'].sum().reset_index(), on="taxonomy", how="left")
tx_filter_summary = tx_filter_summary.merge(tx_df3.groupby('taxonomy')['zero_values'].sum().reset_index(), on="taxonomy", how="left")
tx_filter_summary['missing_values_pct'] = tx_filter_summary['missing_values']/tx_filter_summary['dt_cnt']
tx_filter_summary['zero_values_pct'] = tx_filter_summary['zero_values']/tx_filter_summary['dt_cnt']

# Identify taxonomies to keep for modeling 
tx_keep = tx_filter_summary[(tx_filter_summary['max_pv'] > max_observations_threshold) & (tx_filter_summary['missing_values_pct'] < missing_values_pct_threshold)]# & (tx_filter_summary['zero_values_pct'] < 0.1)]
print(tx_keep.shape)

# 86 out of 136 taxonomies retained for modeling
# 50 out of 136 of taxonomies filtered out

In [ ]:
# Dataset with taxonomies to keep
tx_df4 = tx_df3[tx_df3['taxonomy'].isin(tx_keep['taxonomy'])][['taxonomy', 'traffic_date', 'page_views']]
print(tx_df4)

In [ ]:
# Loop through each taxonomy and fill in missing values using linear interpolation
tx_df4['page_views_impute'] = tx_df4['page_views']

for tx in tx_df4['taxonomy'].unique():
    tx_df4.loc[tx_df4['taxonomy'] == tx, 'page_views_impute'] = tx_df4.loc[tx_df4['taxonomy'] == tx, 'page_views_impute'].interpolate(method='linear')

print(tx_df4[tx_df4['page_views'].isna() == True]['taxonomy'].value_counts())

In [ ]:
# View some taxonomy time series
tx = 'workwear>cooling clothing & gear'

plt.figure(figsize=(12,5))
ax = plt.axes()
ax.set_facecolor("whitesmoke")
plt.title(f"Page Views by Date {tx}")
plt.plot(tx_df4[tx_df4['taxonomy'] == tx]['traffic_date'], tx_df4[tx_df4['taxonomy'] == tx]['page_views_impute'], linewidth=1.5, color="black")
plt.plot(tx_df4[tx_df4['taxonomy'] == tx]['traffic_date'], tx_df4[tx_df4['taxonomy'] == tx]['page_views'], linewidth=0.5, color="lightblue")
plt.xticks(rotation = 45, fontsize=8)
plt.grid(color="white")
plt.show()

In [ ]:
# View aggregated time series
agg_df = tx_df4.groupby('traffic_date')['page_views_impute'].sum().reset_index()

plt.figure(figsize=(12,5))
ax = plt.axes()
ax.set_facecolor("whitesmoke")
plt.title('Page Views by Date')
plt.plot(agg_df['page_views_impute'], linewidth=0.5, color="darkblue")
plt.xticks(rotation = 45, fontsize=8)
plt.grid(color="white")
plt.show()

In [ ]:
# What if page_views are aggregated by week?
agg_df['traffic_week'] = agg_df['traffic_date'].dt.to_period('W').dt.start_time

agg_df2 = agg_df.groupby('traffic_week')['page_views_impute'].sum().reset_index()

plt.figure(figsize=(12,5))
ax = plt.axes()
ax.set_facecolor("whitesmoke")
plt.title('Page Views by Week')
plt.plot(agg_df2['traffic_week'], agg_df2['page_views_impute'], linewidth=0.5, color="darkblue")
plt.xticks(rotation = 45, fontsize=8)
plt.grid(color="white")
plt.show()

# Less noise
# Stick with daily data

In [ ]:
# Iterate through each taxonomy
# Idenfity anomalies using Isolation Forest from sklearn
# Replace them with linear interpolation imputations
# Box-Cox transformation of resulting time series

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy.stats import boxcox
from scipy.special import inv_boxcox

# Set outliers fraction to 0.01, or 1% of the data, which is about 4 observations per year
outliers_fraction = float(0.01)

tx_df4['anomaly'] = 0
tx_df4['page_views_impute_anomaly'] = tx_df4['page_views_impute']
tx_df4['pv_ia_bc'] = 0

boxcox_lambdas_df = pd.DataFrame(tx_df4['taxonomy'].unique(), columns={"taxonomy"})
boxcox_lambdas_df['boxcox_lambda'] = 0

for tx in tx_df4['taxonomy'].unique():
    page_views = tx_df4.loc[tx_df4['taxonomy'] == tx, 'page_views_impute']
    page_views.index = tx_df4.loc[tx_df4['taxonomy'] == tx, 'traffic_date']
    scaler = StandardScaler()

    pv_scaled = scaler.fit_transform(page_views.values.reshape(-1, 1))
    pv_scaled_df = pd.DataFrame(pv_scaled)

    # train isolation forest
    model =  IsolationForest(contamination=outliers_fraction)
    model.fit(pv_scaled_df)

    page_views = pd.DataFrame(page_views)
    page_views['anomaly'] = model.predict(pv_scaled_df)

    page_views = page_views.reset_index()
    page_views.index = tx_df4[tx_df4['taxonomy'] == tx].index

    # Interpolate new value for anomalies
    # Identify anomalies in tx_df4
    tx_df4.loc[tx_df4['taxonomy'] == tx, 'anomaly'] = page_views['anomaly']
    # Set page_views_impute to na where anomaly = -1
    tx_df4.loc[(tx_df4['taxonomy'] == tx) & (tx_df4['anomaly'] == -1), 'page_views_impute_anomaly'] = np.nan
    # Interpolate new nans
    tx_df4.loc[tx_df4['taxonomy'] == tx, 'page_views_impute_anomaly'] = tx_df4.loc[tx_df4['taxonomy'] == tx, 'page_views_impute_anomaly'].interpolate(method='linear')
    # Box-Cox transformation
    tx_df4.loc[tx_df4['taxonomy'] == tx, 'pv_ia_bc'], boxcox_lambdas_df.loc[boxcox_lambdas_df['taxonomy'] == tx, 'boxcox_lambda'] = boxcox(tx_df4.loc[tx_df4['taxonomy'] == tx, 'page_views_impute_anomaly'] + 1)
    
    # visualization
    plt.figure(figsize=(12,5))
    ax = plt.axes()
    ax.set_facecolor("whitesmoke")
    plt.title(f"Page View Anomalies {tx}")
    plt.plot(tx_df4.loc[tx_df4['taxonomy'] == tx, 'traffic_date'], tx_df4.loc[tx_df4['taxonomy'] == tx, 'page_views_impute'], linewidth=0.5, color="lightblue", label='Normal')
    plt.plot(tx_df4.loc[tx_df4['taxonomy'] == tx, 'traffic_date'], tx_df4.loc[tx_df4['taxonomy'] == tx, 'page_views_impute_anomaly'], linewidth=0.5, color="darkblue", label='Normal')
    plt.scatter(tx_df4.loc[(tx_df4['taxonomy'] == tx) & (tx_df4['anomaly'] == -1), 'traffic_date'], tx_df4.loc[(tx_df4['taxonomy'] == tx) & (tx_df4['anomaly'] == -1), 'page_views_impute'], color="red", label='Anomaly')
    plt.xticks(rotation = 45, fontsize=8)
    plt.grid(color="white")
    plt.show()

#     # visualization
#     plt.figure(figsize=(12,5))
#     ax = plt.axes()
#     ax.set_facecolor("whitesmoke")
#     plt.title(f"Page View Box-Cox {tx}")
#     plt.plot(tx_df4.loc[tx_df4['taxonomy'] == tx, 'traffic_date'], tx_df4.loc[tx_df4['taxonomy'] == tx, 'pv_ia_bc'], linewidth=0.5, color="blue")
#     plt.xticks(rotation = 45, fontsize=8)
#     plt.grid(color="white")
#     plt.show()

# Looks reasonable

# Some page_views_impute_anomaly are NA and page_views_impute are not NA
# In those cases, set page_views_impute_anomaly = page_views_impute
tx_df4.loc[(tx_df4['page_views_impute_anomaly'].isna() == True) & (tx_df4['page_views_impute'].isna() == False), 'page_views_impute_anomaly'] = tx_df4.loc[(tx_df4['page_views_impute_anomaly'].isna() == True) & (tx_df4['page_views_impute'].isna() == False), 'page_views_impute']

In [ ]:
# # Validate inverse boxcox function
# tx_df4['pv_ia_bc_inv'] = 0

# for tx in tx_df4['taxonomy'].unique():
#     lmbda = float(boxcox_lambdas_df.loc[boxcox_lambdas_df['taxonomy'] == tx, 'boxcox_lambda'])
#     tx_df4.loc[tx_df4['taxonomy'] == tx, 'pv_ia_bc_inv'] = inv_boxcox(tx_df4.loc[tx_df4['taxonomy'] == tx, 'pv_ia_bc'], lmbda) - 1

# # It works

In [ ]:
# Add engineered date features
tx_df_fe = tx_df4
tx_df_fe['year'] = tx_df_fe['traffic_date'].dt.year
tx_df_fe['quarter'] = tx_df_fe['traffic_date'].dt.quarter
tx_df_fe['month'] = tx_df_fe['traffic_date'].dt.month
tx_df_fe['half_month'] = tx_df_fe['month'] * 2
tx_df_fe.loc[tx_df_fe['traffic_date'].dt.day <= 15, 'half_month'] = tx_df_fe.loc[tx_df_fe['traffic_date'].dt.day <= 15, 'half_month'] - 1
tx_df_fe['dayofweek'] = tx_df_fe['traffic_date'].dt.dayofweek
tx_df_fe['weekend'] = 0
tx_df_fe.loc[tx_df_fe['dayofweek'] >= 5, 'weekend'] = 1

next_xmas_year = tx_df_fe['traffic_date'].dt.year
next_xmas_year.loc[tx_df_fe['traffic_date'] > pd.to_datetime(dict(year=tx_df_fe['traffic_date'].dt.year, month=12, day=25))] = next_xmas_year.loc[tx_df_fe['traffic_date'] > pd.to_datetime(dict(year=tx_df_fe['traffic_date'].dt.year, month=12, day=25))] + 1

tx_df_fe['days_until_xmas'] = (pd.to_datetime(dict(year=next_xmas_year, month=12, day=25)) - tx_df_fe['traffic_date']).dt.days

next_jul4_year = tx_df_fe['traffic_date'].dt.year
next_jul4_year.loc[tx_df_fe['traffic_date'] > pd.to_datetime(dict(year=tx_df_fe['traffic_date'].dt.year, month=7, day=4))] = next_jul4_year.loc[tx_df_fe['traffic_date'] > pd.to_datetime(dict(year=tx_df_fe['traffic_date'].dt.year, month=7, day=4))] + 1

tx_df_fe['days_until_jul4'] = (pd.to_datetime(dict(year=next_jul4_year, month=7, day=4)) - tx_df_fe['traffic_date']).dt.days

print(tx_df_fe)

In [ ]:
# Add engineered page view features
# 7-day MA
# 30-day MA
# Lag 1, 7 and 30

tx_df_fe['pv_ia_lag1'] = 0
tx_df_fe['pv_ia_lag7'] = 0
tx_df_fe['pv_ia_lag30'] = 0

tx_df_fe['pv_ia_ma7'] = 0
tx_df_fe['pv_ia_ma30'] = 0

tx_df_fe['pv_ia_ma7_lag1'] = 0
tx_df_fe['pv_ia_ma7_lag7'] = 0
tx_df_fe['pv_ia_ma7_lag30'] = 0
tx_df_fe['pv_ia_ma30_lag1'] = 0
tx_df_fe['pv_ia_ma30_lag7'] = 0
tx_df_fe['pv_ia_ma30_lag30'] = 0


for tx in tx_df_fe['taxonomy'].unique():
    tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_lag1'] = tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'page_views_impute_anomaly'].shift(1)
    tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_lag7'] = tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'page_views_impute_anomaly'].shift(7)
    tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_lag30'] = tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'page_views_impute_anomaly'].shift(30)
    tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma7'] = tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'page_views_impute_anomaly'].rolling(window=7, min_periods=1).mean()
    tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma30'] = tx_df4.loc[tx_df4['taxonomy'] == tx, 'page_views_impute_anomaly'].rolling(window=30, min_periods=1).mean()    
    tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma7_lag1'] = tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma7'].shift(1)
    tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma7_lag7'] = tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma7'].shift(7)
    tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma7_lag30'] = tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma7'].shift(30)
    tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma30_lag1'] = tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma30'].shift(1)
    tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma30_lag7'] = tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma30'].shift(7)
    tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma30_lag30'] = tx_df_fe.loc[tx_df4['taxonomy'] == tx, 'pv_ia_ma30'].shift(30)

    
    plt.figure(figsize=(12,5))
    ax = plt.axes()
    ax.set_facecolor("whitesmoke")
    plt.title(f"Page View Features {tx}")
    plt.plot(tx_df_fe.loc[tx_df_fe['taxonomy'] == tx, 'traffic_date'], tx_df_fe.loc[tx_df_fe['taxonomy'] == tx, 'page_views_impute_anomaly'], color="blue", linewidth=0.5)
    plt.plot(tx_df_fe.loc[tx_df_fe['taxonomy'] == tx, 'traffic_date'], tx_df_fe.loc[tx_df_fe['taxonomy'] == tx, 'pv_ia_ma7'], color="black")
    plt.plot(tx_df_fe.loc[tx_df_fe['taxonomy'] == tx, 'traffic_date'], tx_df_fe.loc[tx_df_fe['taxonomy'] == tx, 'pv_ia_ma30'], color="red")
    plt.xticks(rotation = 45, fontsize=8)
    plt.grid(color="white")
    plt.show()

print(tx_df4)

In [ ]:
# Test lightgbm forecasting
import lightgbm as lgb
from mlforecast import MLForecast

model = lgb.LGBMRegressor(random_state=50)


gbm_features = ['traffic_date',
                'page_views_impute_anomaly',
                'pv_ia_ma7',
                'pv_ia_ma30',
                'year',
                'quarter',
                'month',
                'dayofweek',
                'weekend',
                'days_until_xmas',
                'days_until_jul4',
                'pv_ia_lag1',
                'pv_ia_lag7',
                'pv_ia_lag30',
                'pv_ia_ma7_lag1',
                'pv_ia_ma7_lag7',
                'pv_ia_ma7_lag30',
                'pv_ia_ma30_lag1',
                'pv_ia_ma30_lag7',
                'pv_ia_ma30_lag30'
               ]

gbm_X = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features[4:]]
gbm_X.index = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features[0]]
gbm_y = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features[1]]
gbm_y.index = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features[0]]

train_X = gbm_X.iloc[:-182, :]
train_y = gbm_y.iloc[:-182]
test_X = gbm_X.iloc[-182:, :]
test_y = gbm_y.iloc[-182:]

# Iterate forecast one day at a time
# Get forecast for next day, then calculate lag features using the forecast

lgbm_fit = lgb.LGBMRegressor(random_state=432, num_leaves=100, min_child_samples=4)
lgbm_fit.fit(train_X, train_y)

lgb.plot_importance(lgbm_fit, importance_type = 'split')
plt.show()

train_y_pred = pd.Series(lgbm_fit.predict(train_X), index=train_y.index)

plt.figure(figsize=(16,5))
ax = plt.axes()
ax.set_facecolor("whitesmoke")
plt.title(f"Page Views Forecast {tx}")
plt.plot(train_y, color = "black", linewidth=2)
plt.plot(train_y_pred, color = "yellow", linewidth=0.5)
plt.xticks(rotation = 45, fontsize=8)
plt.grid(color="white")
plt.show()

print(f"Training MdAPE: {(abs(train_y_pred/train_y - 1)).median()}")

In [ ]:
history = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features].set_index('traffic_date')
history['train_test'] = 'train'
history.loc[history.index >= test_X.index[0], 'train_test'] = 'test'
history['pv_ia_pred'] = 0
history.loc[history['train_test'] == 'train', 'pv_ia_pred'] = train_y_pred
history['pv_ia_history'] = history['page_views_impute_anomaly']

for i in history[history['train_test'] == 'test'].index:
    # Populate lagged input features
    history.loc[history.index == i, "pv_ia_lag1"] = history.loc[history.index == i - pd.Timedelta(days=1), "pv_ia_history"][0]
    history.loc[history.index == i, "pv_ia_lag7"] = history.loc[history.index == i - pd.Timedelta(days=7), "pv_ia_history"][0]
    history.loc[history.index == i, "pv_ia_lag30"] = history.loc[history.index == i - pd.Timedelta(days=30), "pv_ia_history"][0]
    history.loc[history.index == i, "pv_ia_ma7_lag1"] = history.loc[history.index == i - pd.Timedelta(days=1), "pv_ia_ma7"][0]
    history.loc[history.index == i, "pv_ia_ma7_lag7"] = history.loc[history.index == i - pd.Timedelta(days=7), "pv_ia_ma7"][0]
    history.loc[history.index == i, "pv_ia_ma7_lag30"] = history.loc[history.index == i - pd.Timedelta(days=30), "pv_ia_ma7"][0]
    history.loc[history.index == i, "pv_ia_ma30_lag1"] = history.loc[history.index == i - pd.Timedelta(days=1), "pv_ia_ma30"][0]
    history.loc[history.index == i, "pv_ia_ma30_lag7"] = history.loc[history.index == i - pd.Timedelta(days=7), "pv_ia_ma30"][0]
    history.loc[history.index == i, "pv_ia_ma30_lag30"] = history.loc[history.index == i - pd.Timedelta(days=30), "pv_ia_ma30"][0]
    
    # Predict next page_view count, populate in pv_ia_history
    y_hat = lgbm_fit.predict(pd.DataFrame(history.loc[history.index == i, gbm_features[4:]]))[0]
    history.loc[history.index == i, "pv_ia_history"] = y_hat
    history.loc[history.index == i, "pv_ia_pred"] = y_hat
    
    # Populate latest ma7 and ma30 with pv_ia_history
    history.loc[history.index == i, "pv_ia_ma7"] = history.loc[(history.index <= i) & (history.index >= i - pd.Timedelta(days=6)), "pv_ia_history"].mean()
    history.loc[history.index == i, "pv_ia_ma30"] = history.loc[(history.index <= i) & (history.index >= i - pd.Timedelta(days=29)), "pv_ia_history"].mean()
    
# Plot actual vs. pred / forecast
plt.figure(figsize=(16,5))
ax = plt.axes()
ax.set_facecolor("whitesmoke")
plt.title(f"Page Views Forecast {tx}")
plt.plot(history['page_views_impute_anomaly'], color = "black", linewidth=2)
plt.plot(history['pv_ia_pred'], color = "yellow", linewidth=0.5)
plt.xticks(rotation = 45, fontsize=8)
plt.grid(color="white")
plt.show()



In [ ]:
# Aggregate to weekly level and get MAPE
history = history.reset_index()
history['traffic_week'] = history['traffic_date'].dt.to_period('W').dt.start_time

history_wk = history.groupby(['traffic_week', 'train_test'])[['page_views_impute_anomaly', 'pv_ia_pred']].sum().reset_index()

plt.figure(figsize=(12,5))
ax = plt.axes()
ax.set_facecolor("whitesmoke")
plt.title(f"Page Views by Week for {tx}")
plt.plot(history_wk['traffic_week'], history_wk['page_views_impute_anomaly'], linewidth=2, color="darkblue")
plt.plot(history_wk['traffic_week'], history_wk['pv_ia_pred'], linewidth=0.5, color="orange")
plt.xticks(rotation = 45, fontsize=8)
plt.grid(color="white")
plt.show()

In [ ]:
# Get weekly MAPE
mape_lgbm = abs(history_wk[history_wk['train_test'] == 'test']['pv_ia_pred']/history_wk[history_wk['train_test'] == 'test']['page_views_impute_anomaly'] - 1).mean()
print(mape_lgbm)

# Get daily MAPE
mape_lgbm_daily = abs(history[history['train_test'] == 'test']['pv_ia_pred']/history[history['train_test'] == 'test']['page_views_impute_anomaly'] - 1).mean()
print(mape_lgbm_daily)

In [ ]:
# Test AutoARIMA
# from pmdarima.arima import AutoARIMA

import pmdarima as pm
from pmdarima import model_selection

y = tx_df_fe[tx_df_fe['taxonomy'] == tx]['pv_ia_bc']
y.index = tx_df_fe[tx_df_fe['taxonomy'] == tx]['traffic_date']

train_X = gbm_X.iloc[:-182, :]
train_y = gbm_y.iloc[:-182]
test_X = gbm_X.iloc[-182:, :]
test_y = gbm_y.iloc[-182:]

y_train = y.iloc[:-182]
y_test = y.iloc[-182:]

forecast_dates = pd.Series(pd.date_range(tx_df_fe['traffic_date'].max(), periods=183)[1:])
print(forecast_dates)

arima_fit = pm.auto_arima(y_train,
                          max_p = 30,
                          max_P = 366,
                          error_action='ignore',
                          trace=True,
                          suppress_warnings=True,
                          maxiter=5,
                          random_state=120)

y_train_pred = pd.Series(arima_fit.predict_in_sample())
y_train_pred.index = y_train.index
y_test_pred = pd.Series(arima_fit.predict(182))
y_test_pred.index = y_test.index

plt.figure(figsize=(12,5))
ax = plt.axes()
ax.set_facecolor("whitesmoke")
plt.title(f"Page Views Forecast {tx}")
plt.plot(y_train, color = "black", linewidth=2)
plt.plot(y_train_pred, color = "blue", linewidth=0.5)
plt.plot(y_test_pred, color = "green", linewidth=0.5)
plt.xticks(rotation = 45, fontsize=8)
plt.grid(color="white")
plt.show()


# Aggregate to weekly level and get MAPE
arima_df = pd.concat([y_test, y_test_pred], axis = 1).reset_index()
arima_df.columns = ['traffic_date', 'pv_ia_bc', 'pv_ia_bc_forecast']

lmbda = float(boxcox_lambdas_df.loc[boxcox_lambdas_df['taxonomy'] == tx, 'boxcox_lambda'])
print(lmbda)
arima_df['pv_ia'] = inv_boxcox(arima_df['pv_ia_bc'], lmbda) - 1
arima_df['pv_ia_forecast'] = inv_boxcox(arima_df['pv_ia_bc_forecast'], lmbda) - 1

print(arima_df)

arima_df['traffic_week'] = arima_df['traffic_date'].dt.to_period('W').dt.start_time

arima_df_wk = arima_df.groupby(['traffic_week'])[['pv_ia', 'pv_ia_forecast']].sum().reset_index()


# Get weekly MAPE
mape_arima = abs(arima_df_wk['pv_ia_forecast']/arima_df_wk['pv_ia'] - 1).mean()
print(mape_arima)

In [ ]:
# Test FB Prophet
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

y = tx_df_fe[tx_df_fe['taxonomy'] == tx]['page_views_impute_anomaly']
y.index = tx_df_fe[tx_df_fe['taxonomy'] == tx]['traffic_date']

y = y.reset_index()
y.columns = ["ds", "y"]

y_train = y.iloc[:-182, :]


prophet_fit = Prophet(daily_seasonality=True)
prophet_fit.fit(y_train)

future = prophet_fit.make_future_dataframe(periods=182)

forecast = prophet_fit.predict(future)

print(forecast[['ds', 'yhat']])

# fig1 = prophet_fit.plot(forecast)
# fig2 = prophet_fit.plot_components(forecast)

# plot_plotly(prophet_fit, forecast)
# plot_components_plotly(prophet_fit, forecast)
y.columns = ['traffic_date', 'page_views_impute_anomaly']

y['yhat'] = forecast['yhat']

y['train_test'] = 'train'

y.loc[y.index >= y.index.max() - 181, 'train_test'] = 'test'

y['traffic_week'] = y['traffic_date'].dt.to_period('W').dt.start_time

y_wk = y.groupby(['traffic_week', 'train_test'])[['page_views_impute_anomaly', 'yhat']].sum().reset_index()


# Get weekly MAPE
mape_prophet = abs(y_wk[y_wk['train_test'] == 'test']['yhat']/y_wk[y_wk['train_test'] == 'test']['page_views_impute_anomaly'] - 1).mean()
print(mape_prophet)

# Get daily MAPE
mape_prophet_daily = abs(y[y['train_test'] == 'test']['yhat']/y[y['train_test'] == 'test']['page_views_impute_anomaly'] - 1).mean()
print(mape_prophet_daily)


In [ ]:
# Prepare dataframe for final forecasts
# To include:
# - taxonomy
# - week_end_date
# - train or test
# - page_views (original)
# - forecast page_views
# - confidence intervals of forecasts
# - forecast model

# Last forecast date is 2024-07-21

actual_days = len(tx_df_fe['traffic_date'].unique())
last_actual_day = tx_df_fe['traffic_date'].max()
traffic_dates = pd.DataFrame(pd.Series(pd.date_range(tx_df_fe['traffic_date'].min(), periods=actual_days + 182))).rename(columns={0: "traffic_date"})
traffic_dates['week_end_date'] = pd.to_datetime(traffic_dates['traffic_date'].dt.to_period('W').dt.end_time.dt.date)
traffic_dates['key'] = 0
traffic_dates['actual_forecast'] = 'actual'
traffic_dates.loc[traffic_dates['traffic_date'] > last_actual_day, 'actual_forecast'] = 'forecast'

txs = pd.DataFrame(pd.Series(tx_df_fe['taxonomy'].unique())).rename(columns={0: 'taxonomy'})
txs['key'] = 0

final_daily_df = txs.merge(traffic_dates, on='key', how='outer').drop(columns='key')
final_daily_df = final_daily_df.merge(tx_df_fe[['taxonomy', 'traffic_date', 'page_views', 'page_views_impute_anomaly']], on=['taxonomy', 'traffic_date'], how='left')

final_daily_df['page_views_forecast'] = np.nan
final_daily_df['page_views_forecast_lower'] = np.nan
final_daily_df['page_views_forecast_upper'] = np.nan
final_daily_df['forecast_model'] = np.nan

final_daily_df['keep'] = 1
final_daily_df.loc[(final_daily_df['actual_forecast'] == 'actual') & (final_daily_df['page_views_impute_anomaly'].isna() == True), 'keep'] = 0
final_daily_df = final_daily_df[final_daily_df['keep'] == 1].drop(columns='keep')

In [ ]:
gbm_features = ['traffic_date',
                'page_views_impute_anomaly',
                'pv_ia_ma7',
                'pv_ia_ma30',
                'year',
                'quarter',
                'month',
                'dayofweek',
                'weekend',
                'days_until_xmas',
                'days_until_jul4',
                'pv_ia_lag1',
                'pv_ia_lag7',
                'pv_ia_lag30',
                'pv_ia_ma7_lag1',
                'pv_ia_ma7_lag7',
                'pv_ia_ma7_lag30',
                'pv_ia_ma30_lag1',
                'pv_ia_ma30_lag7',
                'pv_ia_ma30_lag30'
               ]

# Function to get weekly test MAPE of LGBM
def lgbm_eval(tx_df_fe: pd.DataFrame, gbm_features: list, tx: str):

    gbm_X = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features[4:]]

    if gbm_X.shape[0] > 365:
        gbm_X.index = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features[0]]
        gbm_y = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features[1]]
        gbm_y.index = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features[0]]

        train_X = gbm_X.iloc[:-182, :]
        train_y = gbm_y.iloc[:-182]
        test_X = gbm_X.iloc[-182:, :]
        test_y = gbm_y.iloc[-182:]

        # Iterate forecast one day at a time
        # Get forecast for next day, then calculate lag features using the forecast

        lgbm_fit = lgb.LGBMRegressor(random_state=432, min_child_samples=4)
        lgbm_fit.fit(train_X, train_y)

        train_y_pred = pd.Series(lgbm_fit.predict(train_X), index=train_y.index)
        
        history = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features].set_index('traffic_date')
        history['train_test'] = 'train'
        history.loc[history.index >= test_X.index.min(), 'train_test'] = 'test'
        history['pv_ia_pred'] = 0
        history.loc[history['train_test'] == 'train', 'pv_ia_pred'] = train_y_pred
        history['pv_ia_history'] = history['page_views_impute_anomaly']

        for i in history[history['train_test'] == 'test'].index:
            # Populate lagged input features
            history.loc[history.index == i, "pv_ia_lag1"] = history.loc[history.index == i - pd.Timedelta(days=1), "pv_ia_history"][0]
            history.loc[history.index == i, "pv_ia_lag7"] = history.loc[history.index == i - pd.Timedelta(days=7), "pv_ia_history"][0]
            history.loc[history.index == i, "pv_ia_lag30"] = history.loc[history.index == i - pd.Timedelta(days=30), "pv_ia_history"][0]
            history.loc[history.index == i, "pv_ia_ma7_lag1"] = history.loc[history.index == i - pd.Timedelta(days=1), "pv_ia_ma7"][0]
            history.loc[history.index == i, "pv_ia_ma7_lag7"] = history.loc[history.index == i - pd.Timedelta(days=7), "pv_ia_ma7"][0]
            history.loc[history.index == i, "pv_ia_ma7_lag30"] = history.loc[history.index == i - pd.Timedelta(days=30), "pv_ia_ma7"][0]
            history.loc[history.index == i, "pv_ia_ma30_lag1"] = history.loc[history.index == i - pd.Timedelta(days=1), "pv_ia_ma30"][0]
            history.loc[history.index == i, "pv_ia_ma30_lag7"] = history.loc[history.index == i - pd.Timedelta(days=7), "pv_ia_ma30"][0]
            history.loc[history.index == i, "pv_ia_ma30_lag30"] = history.loc[history.index == i - pd.Timedelta(days=30), "pv_ia_ma30"][0]

            # Predict next page_view count, populate in pv_ia_history
            y_hat = lgbm_fit.predict(pd.DataFrame(history.loc[history.index == i, gbm_features[4:]]))[0]
            history.loc[history.index == i, "pv_ia_history"] = y_hat
            history.loc[history.index == i, "pv_ia_pred"] = y_hat

            # Populate latest ma7 and ma30 with pv_ia_history
            history.loc[history.index == i, "pv_ia_ma7"] = history.loc[(history.index <= i) & (history.index >= i - pd.Timedelta(days=6)), "pv_ia_history"].mean()
            history.loc[history.index == i, "pv_ia_ma30"] = history.loc[(history.index <= i) & (history.index >= i - pd.Timedelta(days=29)), "pv_ia_history"].mean()

        history = history.reset_index()
        history['traffic_week'] = history['traffic_date'].dt.to_period('W').dt.start_time

        history_wk = history.groupby(['traffic_week', 'train_test'])[['page_views_impute_anomaly', 'pv_ia_pred']].sum().reset_index()

        # Get weekly MAPE
        mape_lgbm = round(abs(history_wk[history_wk['train_test'] == 'test']['pv_ia_pred']/history_wk[history_wk['train_test'] == 'test']['page_views_impute_anomaly'] - 1).mean(), 5)
    else:
        mape_lgbm = np.inf
    return mape_lgbm

# Function to get final LGBM forecasts
def lgbm_final(tx_df_fe: pd.DataFrame, gbm_features: list, tx: str, final_daily_df: pd.DataFrame):

    gbm_features_trim = gbm_features.copy()
    gbm_features_trim.remove('page_views_impute_anomaly')
    
    gbm_X = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features[4:]]
    gbm_X.index = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features[0]]
    gbm_y = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features[1]]
    gbm_y.index = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features[0]]

    # Iterate forecast one day at a time
    # Get forecast for next day, then calculate lag features using the forecast

    lgbm_fit = lgb.LGBMRegressor(random_state=432, min_child_samples=4)
    lgbm_fit.fit(gbm_X, gbm_y)

    lgbm_fit_lb = lgb.LGBMRegressor(random_state=396, min_child_samples_=4, objective = 'quantile', alpha = 0.1)
    lgbm_fit_lb.fit(gbm_X, gbm_y)

    lgbm_fit_ub = lgb.LGBMRegressor(random_state=396, min_child_samples_=4, objective = 'quantile', alpha = 0.9)
    lgbm_fit_ub.fit(gbm_X, gbm_y)

    gbm_y_pred = pd.Series(lgbm_fit.predict(gbm_X), index=gbm_y.index)
    gbm_y_pred_lb = pd.Series(lgbm_fit_lb.predict(gbm_X), index=gbm_y.index)
    gbm_y_pred_ub = pd.Series(lgbm_fit_ub.predict(gbm_X), index=gbm_y.index)

    hist_index = final_daily_df.loc[final_daily_df['taxonomy'] == tx, 'traffic_date']
    hist_df1 = final_daily_df[final_daily_df['taxonomy'] == tx].reset_index(drop=True)
    hist_df2 = tx_df_fe[tx_df_fe['taxonomy'] == tx][gbm_features_trim].reset_index(drop=True)
    history = hist_df1.merge(hist_df2, on=['traffic_date'], how='left').set_index(hist_index)

    history['pv_ia_history'] = history['page_views_impute_anomaly']
    history.loc[history['actual_forecast'] == 'actual', 'page_views_forecast'] = gbm_y_pred
    history.loc[history['actual_forecast'] == 'actual', 'page_views_forecast_lower'] = gbm_y_pred_lb
    history.loc[history['actual_forecast'] == 'actual', 'page_views_forecast_upper'] = gbm_y_pred_ub


    for i in history[history['actual_forecast'] == 'forecast']['traffic_date']:

        # Populate lagged input features
        history.loc[history.index == i, "pv_ia_lag1"] = history.loc[history.index == i - pd.Timedelta(days=1), "pv_ia_history"][0]
        history.loc[history.index == i, "pv_ia_lag7"] = history.loc[history.index == i - pd.Timedelta(days=7), "pv_ia_history"][0]
        history.loc[history.index == i, "pv_ia_lag30"] = history.loc[history.index == i - pd.Timedelta(days=30), "pv_ia_history"][0]
        history.loc[history.index == i, "pv_ia_ma7_lag1"] = history.loc[history.index == i - pd.Timedelta(days=1), "pv_ia_ma7"][0]
        history.loc[history.index == i, "pv_ia_ma7_lag7"] = history.loc[history.index == i - pd.Timedelta(days=7), "pv_ia_ma7"][0]
        history.loc[history.index == i, "pv_ia_ma7_lag30"] = history.loc[history.index == i - pd.Timedelta(days=30), "pv_ia_ma7"][0]
        history.loc[history.index == i, "pv_ia_ma30_lag1"] = history.loc[history.index == i - pd.Timedelta(days=1), "pv_ia_ma30"][0]
        history.loc[history.index == i, "pv_ia_ma30_lag7"] = history.loc[history.index == i - pd.Timedelta(days=7), "pv_ia_ma30"][0]
        history.loc[history.index == i, "pv_ia_ma30_lag30"] = history.loc[history.index == i - pd.Timedelta(days=30), "pv_ia_ma30"][0]

        # Predict next page_view count, populate in pv_ia_history
        y_hat = lgbm_fit.predict(pd.DataFrame(history.loc[history.index == i, gbm_features[4:]]))[0]
        history.loc[history.index == i, "pv_ia_history"] = y_hat
        history.loc[history.index == i, "page_views_forecast"] = y_hat
        y_hat_lower = lgbm_fit_lb.predict(pd.DataFrame(history.loc[history.index == i, gbm_features[4:]]))[0]
        history.loc[history.index == i, "page_views_forecast_lower"] = y_hat_lower
        y_hat_upper = lgbm_fit_ub.predict(pd.DataFrame(history.loc[history.index == i, gbm_features[4:]]))[0]
        history.loc[history.index == i, "page_views_forecast_upper"] = y_hat_upper

        # Populate latest ma7 and ma30 with pv_ia_history
        history.loc[history.index == i, "pv_ia_ma7"] = history.loc[(history.index <= i) & (history.index >= i - pd.Timedelta(days=6)), "pv_ia_history"].mean()
        history.loc[history.index == i, "pv_ia_ma30"] = history.loc[(history.index <= i) & (history.index >= i - pd.Timedelta(days=29)), "pv_ia_history"].mean()

    history.index = final_daily_df[final_daily_df['taxonomy'] == tx].index

    # Add taxonomy data to final_daily_df
    final_daily_df.loc[final_daily_df['taxonomy'] == tx, 'page_views_forecast'] = history['page_views_forecast']
    final_daily_df.loc[final_daily_df['taxonomy'] == tx, 'page_views_forecast_lower'] = history['page_views_forecast_lower']
    final_daily_df.loc[final_daily_df['taxonomy'] == tx, 'page_views_forecast_upper'] = history['page_views_forecast_upper']
    final_daily_df.loc[final_daily_df['taxonomy'] == tx, 'forecast_model'] = 'LGBM'

    return final_daily_df

In [ ]:
# Function to get Auto ARIMA MAPE
def arima_eval(tx_df_fe: pd.DataFrame, boxcox_lambdas_df: pd.DataFrame, tx: str):

    y = tx_df_fe[tx_df_fe['taxonomy'] == tx]['pv_ia_bc']
    if y.shape[0] >= 365:
        y.index = tx_df_fe[tx_df_fe['taxonomy'] == tx]['traffic_date']

        y_train = y.iloc[:-182]
        y_test = y.iloc[-182:]

        arima_fit = pm.auto_arima(y_train,
                                  max_p = 30,
                                  max_P = 366,
                                  error_action='ignore',
                                  trace=True,
                                  suppress_warnings=True,
                                  maxiter=5,
                                  random_state=120)

        y_train_pred = pd.Series(arima_fit.predict_in_sample())
        y_train_pred.index = y_train.index
        y_test_pred = pd.Series(arima_fit.predict(182))
        y_test_pred.index = y_test.index

        # Aggregate to weekly level and get MAPE
        lmbda = float(boxcox_lambdas_df.loc[boxcox_lambdas_df['taxonomy'] == tx, 'boxcox_lambda'])

        arima_df = pd.concat([y_test, y_test_pred], axis = 1).reset_index()
        arima_df.columns = ['traffic_date', 'pv_ia_bc', 'pv_ia_bc_forecast']
        arima_df['pv_ia'] = inv_boxcox(arima_df['pv_ia_bc'], lmbda) - 1
        arima_df['pv_ia_forecast'] = inv_boxcox(arima_df['pv_ia_bc_forecast'], lmbda) - 1
        arima_df['traffic_week'] = arima_df['traffic_date'].dt.to_period('W').dt.start_time
        arima_df_wk = arima_df.groupby(['traffic_week'])[['pv_ia', 'pv_ia_forecast']].sum().reset_index()

        # Get weekly MAPE
        mape_arima = round(abs(arima_df_wk['pv_ia_forecast']/arima_df_wk['pv_ia'] - 1).mean(), 5)
        
    else:
        mape_arima = np.inf
    
    return mape_arima


# Function to get final LGBM forecasts
def arima_final(tx_df_fe: pd.DataFrame, boxcox_lambdas_df: pd.DataFrame, tx: str, final_daily_df: pd.DataFrame):
    
    lmbda = float(boxcox_lambdas_df.loc[boxcox_lambdas_df['taxonomy'] == tx, 'boxcox_lambda'])
    
    y = tx_df_fe[tx_df_fe['taxonomy'] == tx]['pv_ia_bc']
    y.index = tx_df_fe[tx_df_fe['taxonomy'] == tx]['traffic_date']

    arima_fit = pm.auto_arima(y,
                              max_p = 30,
                              max_P = 366,
                              error_action='ignore',
                              trace=True,
                              suppress_warnings=True,
                              maxiter=5,
                              random_state=120)

    y_pred = (arima_fit.predict_in_sample(return_conf_int=True, alpha=0.2))
    y_pred = pd.concat([pd.DataFrame(y_pred[0]), pd.DataFrame(y_pred[1])], axis=1)
    y_pred.columns = ['page_views_forecast', 'page_views_forecast_lower', 'page_views_forecast_upper']
    for col in y_pred.columns:
        y_pred[col] = inv_boxcox(y_pred[col], lmbda) - 1
    y_pred.index = y.index
    y_pred = y_pred.reset_index()
    y_pred.index = final_daily_df[(final_daily_df['taxonomy'] == tx) & (final_daily_df['actual_forecast'] == 'actual')].index

    y_forecast = (arima_fit.predict(n_periods=182, return_conf_int=True, alpha=0.2))
    y_forecast = pd.concat([pd.DataFrame(y_forecast[0]), pd.DataFrame(y_forecast[1])], axis=1)
    y_forecast.columns = ['page_views_forecast', 'page_views_forecast_lower', 'page_views_forecast_upper']
    for col in y_forecast.columns:
        y_forecast[col] = inv_boxcox(y_forecast[col], lmbda) - 1
    forecast_dates = pd.Series(pd.date_range(final_daily_df[final_daily_df['actual_forecast'] == 'forecast']['traffic_date'].min(), periods=182))
    y_forecast['traffic_date'] = forecast_dates
    y_forecast.index = final_daily_df[(final_daily_df['taxonomy'] == tx) & (final_daily_df['actual_forecast'] == 'forecast')].index

    final_daily_df.loc[(final_daily_df['taxonomy'] == tx) & (final_daily_df['actual_forecast'] == 'actual'), 'page_views_forecast'] = y_pred['page_views_forecast']
    final_daily_df.loc[(final_daily_df['taxonomy'] == tx) & (final_daily_df['actual_forecast'] == 'actual'), 'page_views_forecast_lower'] = y_pred['page_views_forecast_lower']
    final_daily_df.loc[(final_daily_df['taxonomy'] == tx) & (final_daily_df['actual_forecast'] == 'actual'), 'page_views_forecast_upper'] = y_pred['page_views_forecast_upper']

    final_daily_df.loc[(final_daily_df['taxonomy'] == tx) & (final_daily_df['actual_forecast'] == 'forecast'), 'page_views_forecast'] = y_forecast['page_views_forecast']
    final_daily_df.loc[(final_daily_df['taxonomy'] == tx) & (final_daily_df['actual_forecast'] == 'forecast'), 'page_views_forecast_lower'] = y_forecast['page_views_forecast_lower']
    final_daily_df.loc[(final_daily_df['taxonomy'] == tx) & (final_daily_df['actual_forecast'] == 'forecast'), 'page_views_forecast_upper'] = y_forecast['page_views_forecast_upper']

    final_daily_df.loc[final_daily_df['taxonomy'] == tx, 'forecast_model'] = 'ARIMA'

    return final_daily_df

In [ ]:
def prophet_eval(tx_df_fe: pd.DataFrame, tx: str):

    y = tx_df_fe[tx_df_fe['taxonomy'] == tx]['page_views_impute_anomaly']
    
    if y.shape[0] >= 365:
        y.index = tx_df_fe[tx_df_fe['taxonomy'] == tx]['traffic_date']
        y = y.reset_index()
        y.columns = ["ds", "y"]

        y_train = y.iloc[:-182, :]

        prophet_fit = Prophet(daily_seasonality=True)
        prophet_fit.fit(y_train)

        future = prophet_fit.make_future_dataframe(periods=182)

        forecast = prophet_fit.predict(future)

        y.columns = ['traffic_date', 'page_views_impute_anomaly']
        y['yhat'] = forecast['yhat']
        y['train_test'] = 'train'
        y.loc[y.index >= y.index.max() - 181, 'train_test'] = 'test'
        y['traffic_week'] = y['traffic_date'].dt.to_period('W').dt.start_time

        y_wk = y.groupby(['traffic_week', 'train_test'])[['page_views_impute_anomaly', 'yhat']].sum().reset_index()

        # Get weekly MAPE
        mape_prophet = round(abs(y_wk[y_wk['train_test'] == 'test']['yhat']/y_wk[y_wk['train_test'] == 'test']['page_views_impute_anomaly'] - 1).mean(), 5)
    
    else:
        mape_prophet = np.inf
    
    return mape_prophet


def prophet_final(tx_df_fe: pd.DataFrame, tx: str, final_daily_df: pd.DataFrame):

    y = tx_df_fe[tx_df_fe['taxonomy'] == tx]['page_views_impute_anomaly']
    y.index = tx_df_fe[tx_df_fe['taxonomy'] == tx]['traffic_date']
    y = y.reset_index()
    y.columns = ["ds", "y"]

    prophet_fit = Prophet(daily_seasonality=True)
    prophet_fit.fit(y)

    future = prophet_fit.make_future_dataframe(periods=182)

    forecast = prophet_fit.predict(future)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    forecast.loc[forecast['yhat_lower'] < 0, 'yhat_lower'] = 0
    forecast.index = final_daily_df[(final_daily_df['taxonomy'] == tx)].index

    final_daily_df.loc[final_daily_df['taxonomy'] == tx, 'page_views_forecast'] = forecast['yhat']
    final_daily_df.loc[final_daily_df['taxonomy'] == tx, 'page_views_forecast_lower'] = forecast['yhat_lower']
    final_daily_df.loc[final_daily_df['taxonomy'] == tx, 'page_views_forecast_upper'] = forecast['yhat_upper']

    final_daily_df.loc[final_daily_df['taxonomy'] == tx, 'forecast_model'] = 'Prophet'
    
    return final_daily_df

In [ ]:
for tx in tx_df_fe['taxonomy'].unique():
    # LightGBM
    try:
        mape_lgbm = lgbm_eval(tx_df_fe, gbm_features, tx)
    except:
        mape_lgbm = np.inf
        
    # Auto ARIMA
    try:
        mape_arima = arima_eval(tx_df_fe, boxcox_lambdas_df, tx)
    except:
        mape_arima = np.inf
    
    # Prophet
    try:
        mape_prophet = prophet_eval(tx_df_fe, tx)
    except:
        mape_prophet = np.inf
    
    print("")
    print("")
    print(f"LGBM MAPE for {tx}: {mape_lgbm}")
    print(f"ARIMA MAPE for {tx}: {mape_arima}")
    print(f"Prophet MAPE for {tx}: {mape_prophet}")
    print("")
    print("")
    
    # Final model based on model with lowest MAPE
    # try:
    if (mape_lgbm <= mape_arima) and (mape_lgbm <= mape_prophet) and (mape_lgbm < np.inf):
        final_daily_df = lgbm_final(tx_df_fe, gbm_features, tx, final_daily_df)
    elif (mape_arima < mape_lgbm) and (mape_arima <= mape_prophet) and (mape_arima < np.inf):
        final_daily_df = arima_final(tx_df_fe, boxcox_lambdas_df, tx, final_daily_df)
    elif (mape_prophet < mape_lgbm) and (mape_prophet < mape_arima) and (mape_prophet < np.inf):
        final_daily_df = prophet_final(tx_df_fe, tx, final_daily_df)
    else:
        print(f"All models failed for {tx}")
    # except:
        # print(f"Failed to get final model and forecasts for {tx}")


In [ ]:
# Plots of each taxonomy actual vs. forecast
for tx in final_daily_df['taxonomy'].unique():

    actual = final_daily_df[(final_daily_df['taxonomy'] == tx) & (final_daily_df["actual_forecast"] == "actual")][["traffic_date", "page_views"]]
    pred = final_daily_df[(final_daily_df['taxonomy'] == tx)  & (final_daily_df["actual_forecast"] == "actual")][["traffic_date", "page_views_forecast"]]
    forecast = final_daily_df[(final_daily_df['taxonomy'] == tx)  & (final_daily_df["actual_forecast"] == "forecast")][["traffic_date", "page_views_forecast"]]
    model = final_daily_df[final_daily_df['taxonomy'] == tx]["forecast_model"].unique()[0]

    plt.figure(figsize=(12,5))
    ax = plt.axes()
    ax.set_facecolor("whitesmoke")
    plt.title(f"Actual and {model} Forecast Page Views for {tx}")
    plt.plot(actual['traffic_date'], actual['page_views'], linewidth=1, color="darkblue")
    plt.plot(pred['traffic_date'], pred['page_views_forecast'], linewidth=1, color="lightblue")
    plt.plot(forecast['traffic_date'], forecast['page_views_forecast'], linewidth=2, color="red")
    plt.xticks(rotation = 45, fontsize=8)
    plt.grid(color="white")
    plt.show()


In [ ]:
# How often was each model the "winner"?
display(final_daily_df[['taxonomy', 'forecast_model']].drop_duplicates()['forecast_model'].value_counts())
print("")
print(final_daily_df[['taxonomy', 'forecast_model']].drop_duplicates().shape)

In [ ]:
# Aggregate to weekly
final_daily_df['upper_pct'] = abs(final_daily_df['page_views_forecast_upper'] - final_daily_df['page_views_forecast'])/final_daily_df['page_views_forecast']
final_daily_df['lower_pct'] = abs(final_daily_df['page_views_forecast_lower'] - final_daily_df['page_views_forecast'])/final_daily_df['page_views_forecast']

weekly_int_pct = final_daily_df.groupby(['taxonomy', 'week_end_date'])[['lower_pct', 'upper_pct']].max().reset_index().replace(np.inf, 2)

final_weekly_df = final_daily_df.groupby(['taxonomy', 'week_end_date', 'forecast_model', 'actual_forecast'])[['page_views', 'page_views_forecast']].sum().reset_index()
final_weekly_df = final_weekly_df.merge(weekly_int_pct, on=['taxonomy', 'week_end_date'], how='left')
final_weekly_df['page_views_forecast_lower'] = final_weekly_df['page_views_forecast'] * (1 - final_weekly_df['lower_pct'])
final_weekly_df['page_views_forecast_upper'] = final_weekly_df['page_views_forecast'] * (1 + final_weekly_df['upper_pct'])
final_weekly_df = final_weekly_df.drop(columns=['lower_pct', 'upper_pct'])

display(final_weekly_df)

In [ ]:
# Plot each weekly taxonomy
for tx in final_weekly_df['taxonomy'].unique():

    actual = final_weekly_df[(final_weekly_df['taxonomy'] == tx) & (final_weekly_df["actual_forecast"] == "actual")][["week_end_date", "page_views"]]
    pred = final_weekly_df[(final_weekly_df['taxonomy'] == tx)  & (final_weekly_df["actual_forecast"] == "actual")][["week_end_date", "page_views_forecast", "page_views_forecast_lower", "page_views_forecast_upper"]]
    forecast = final_weekly_df[(final_weekly_df['taxonomy'] == tx)  & (final_weekly_df["actual_forecast"] == "forecast")][["week_end_date", "page_views_forecast", "page_views_forecast_lower", "page_views_forecast_upper"]]
    model = final_weekly_df[final_weekly_df['taxonomy'] == tx]["forecast_model"].unique()[0]
    
    plt.figure(figsize=(12,5))
    ax = plt.axes()
    ax.set_facecolor("whitesmoke")
    plt.title(f"Weekly Actual and {model} Forecast Page Views for {tx}")
    plt.plot(actual['week_end_date'], actual['page_views'], linewidth=1, color="darkblue")
    plt.plot(pred['week_end_date'], pred['page_views_forecast'], linewidth=1, color="lightblue")
    plt.plot(forecast['week_end_date'], forecast['page_views_forecast_lower'], linewidth=2, color="orange")
    plt.plot(forecast['week_end_date'], forecast['page_views_forecast'], linewidth=2, color="red")
    plt.plot(forecast['week_end_date'], forecast['page_views_forecast_upper'], linewidth=2, color="orange")
    plt.xticks(rotation = 45, fontsize=8)
    plt.grid(color="white")
    plt.show()

In [ ]:
final_weekly_df.to_csv("Quant Research Forecasts JH.csv", index=False)